# 📝 Reddit 규칙 위반 탐지 앙상블 모델 추론

이 노트북은 사전에 훈련된 **Cross-Encoder + LGBM 앙상블 모델**을 불러와 `test.csv` 데이터에 대한 예측을 수행하고, `submission.csv` 파일을 생성합니다.

### **✅ 실행 전 확인 사항**
1. **인터넷(Internet) OFF**: 노트북 설정에서 인터넷이 꺼져있는지 확인하세요.
2. **데이터셋 추가**:
    - **모델 데이터셋**: `model_output` 폴더가 포함된 Kaggle 데이터셋 (예: `my-rule-violation-ensemble-model`)
    - **원본 데이터셋**: `test.csv`가 포함된 대회 데이터

In [ ]:
import os
import pickle
import joblib
import pandas as pd
import numpy as np
import re
from scipy.sparse import hstack
from tqdm import tqdm
import torch
from sentence_transformers import CrossEncoder

# 경고 메시지 무시
import warnings
warnings.filterwarnings('ignore')

print("✅ 라이브러리 임포트 완료")

## 1단계: 모델 및 전처리 객체 로드

훈련 시 저장했던 모든 구성 요소(Cross-Encoder, LGBM 모델, Scaler, OneHotEncoder, 컬럼 리스트)를 불러옵니다.

In [ ]:
# --- 모델 및 객체가 저장된 경로 설정 ---
# 본인의 Kaggle 데이터셋 경로에 맞게 수정해주세요.
MODEL_PATH = '/kaggle/input/my-rule-violation-ensemble-model/model_output/'

print(f"모델 경로: {MODEL_PATH}")

# 1. Cross-Encoder 모델 로드
cross_encoder_path = os.path.join(MODEL_PATH, 'final_cross_encoder_model')
cross_encoder_model = CrossEncoder(cross_encoder_path)
print("   - Cross-Encoder 모델 로드 완료")

# 2. LGBM 모델 로드
lgbm_model = joblib.load(os.path.join(MODEL_PATH, 'lgbm_model.pkl'))
print("   - LGBM 모델 로드 완료")

# 3. Scaler 및 OneHotEncoder 로드
scaler = joblib.load(os.path.join(MODEL_PATH, 'scaler.pkl'))
onehot_encoder = joblib.load(os.path.join(MODEL_PATH, 'onehot_encoder.pkl'))
print("   - Scaler 및 OneHotEncoder 로드 완료")

# 4. 수치형 컬럼 리스트 로드
with open(os.path.join(MODEL_PATH, 'numerical_cols.pkl'), 'rb') as f:
    numerical_cols = pickle.load(f)
print("   - 수치형 컬럼 리스트 로드 완료")

print("\n✅ 모든 모델 및 전처리 객체 로딩 완료!")

## 2단계: 데이터 준비 및 특징 엔지니어링

`test.csv`를 불러오고, 훈련 과정과 동일한 특징 엔지니어링 함수들을 정의하고 적용합니다.

In [ ]:
# --- 특징 엔지니어링 함수들 (훈련 노트북과 동일) ---
def count_urls(text):
    return len(re.findall(r'https?://\S+|www\.\S+', str(text)))
def count_exclaims(text):
    return str(text).count('!')
def count_questions(text):
    return str(text).count('?')
def upper_ratio(text):
    s = str(text)
    letters = [c for c in s if c.isalpha()]
    if not letters: return 0.0
    return sum(1 for c in letters if c.isupper()) / len(letters)
def repeat_char_max(text):
    longest = 1
    last, cur = '', 0
    for ch in str(text):
        if ch == last: cur += 1
        else: longest, cur, last = max(longest, cur), 1, ch
    return max(longest, cur)
def jaccard_similarity(text1, text2):
    set1 = set(str(text1).lower().split())
    set2 = set(str(text2).lower().split())
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union > 0 else 0.0

def create_features(df):
    df = df.copy()
    df['body_len'] = df['body'].astype(str).str.len()
    df['rule_len'] = df['rule'].astype(str).str.len()
    df['body_words'] = df['body'].astype(str).str.split().str.len()
    df['url_cnt'] = df['body'].apply(count_urls)
    df['exc_cnt'] = df['body'].apply(count_exclaims)
    df['q_cnt'] = df['body'].apply(count_questions)
    df['upper_rt'] = df['body'].apply(upper_ratio)
    df['rep_run'] = df['body'].apply(repeat_char_max)
    df['rule_body_jaccard'] = [jaccard_similarity(r, b) for r, b in zip(df['rule'], df['body'])]
    return df

def prepare_cross_encoder_input(rule, body, positive_ex1=None, negative_ex1=None):
    rule_text, comment_text = str(rule).strip(), str(body).strip()
    examples_text = ""
    if pd.notna(positive_ex1) and str(positive_ex1).strip():
        examples_text += f" [긍정예시] {str(positive_ex1).strip()}"
    if pd.notna(negative_ex1) and str(negative_ex1).strip():
        examples_text += f" [부정예시] {str(negative_ex1).strip()}"
    return f"[규칙] {rule_text}{examples_text} [댓글] {comment_text}"

# --- test.csv 로드 및 전처리 실행 ---
test_df = pd.read_csv('/kaggle/input/reddit-rule-violation-prediction/test.csv')
print("test.csv 로드 완료.")

test_df_featured = create_features(test_df)
print("특징 엔지니어링 완료.")

## 3단계: 예측 실행 및 제출 파일 생성

전체 파이프라인을 순서대로 실행하여 최종 예측 확률을 계산하고 `submission.csv` 파일을 생성합니다.

In [ ]:
# --- 1. Cross-Encoder 입력 준비 및 예측 ---
print("1/4: Cross-Encoder 입력 준비 중...")
test_ce_inputs = []
for row in tqdm(test_df.itertuples(), total=len(test_df)):
    test_ce_inputs.append(prepare_cross_encoder_input(
        row.rule, row.body, 
        getattr(row, 'positive_example_1', None), 
        getattr(row, 'negative_example_1', None)
    ))

test_predict_examples = []
for ce_input in test_ce_inputs:
    if '[댓글]' in ce_input:
        rule_part, comment_part = ce_input.split('[댓글]', 1)
        test_predict_examples.append([rule_part.strip(), comment_part.strip()])
    else:
        test_predict_examples.append([ce_input.strip(), ""])

print("2/4: Cross-Encoder로 semantic score 예측 중...")
test_ce_scores = cross_encoder_model.predict(test_predict_examples, show_progress_bar=True, batch_size=64)
test_df_featured['ce_score'] = test_ce_scores

# --- 2. LGBM 입력 준비 ---
print("3/4: LGBM 입력을 위한 데이터 변환 중...")
test_numerical_features = scaler.transform(test_df_featured[numerical_cols])
test_categorical_features = onehot_encoder.transform(test_df_featured[['subreddit']])
X_test_lgbm = hstack([test_numerical_features, test_categorical_features])

# --- 3. 최종 예측 및 제출 파일 생성 ---
print("4/4: 최종 예측 및 제출 파일 생성 중...")
final_predictions_proba = lgbm_model.predict_proba(X_test_lgbm)[:, 1]

submission_df = pd.DataFrame({
    'row_id': test_df['row_id'],
    'rule_violation': final_predictions_proba
})

submission_df.to_csv('submission.csv', index=False)

print("\n🎉 모든 작업 완료! `submission.csv` 파일이 생성되었습니다.")
print("생성된 파일 샘플:")
display(submission_df.head())